In [1]:
print("hi all!")

hi all!


In [2]:
#Google Sheet Link: https://docs.google.com/spreadsheets/d/1RiU9c3YihCsAqh18C25WMfTxkOiAf-nXhxAWtEaKLA0/edit#gid=0
import pandas as pd

sheet_id = '1RiU9c3YihCsAqh18C25WMfTxkOiAf-nXhxAWtEaKLA0'
xls = pd.ExcelFile(f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=xlsx")

arrivals = pd.read_excel(xls, 'Arrivals', header = 1)
departures = pd.read_excel(xls, 'Departures', header = 1)



,Phoenix SkyHarbor,DAL,810.0,N10SDX,DTW,PHX,8:54 AM EST,11:13 AM MST,8.0,126.6,10:51 AM MST,Just Outside City of PHX,Pilot,Greeting,"""Approach Delta 810. Descending down to 7000.""",Unnamed: 15,Greeting.1
0,NaN,DAL,810.0,N10SDX,DTW,PHX,8:54 AM EST,11:13 AM MST,8.0,126.6,10:51 AM MST,Just Outside City of PHX,ATC,Greeting,"""Delta 810. Phoenix Approach. Good morning.""",NaN,Greeting
1,NaN,DAL,810.0,N10SDX,DTW,PHX,8:54 AM EST,11:13 AM MST,8.0,126.6,10:54 AM MST,"Inside City of PHX, North of Airport",ATC,Direction to Descend,"""Delta 810, descend to maintain 4000.""",NaN,Direction to Descend
2,NaN,DAL,810.0,N10SDX,DTW,PHX,8:54 AM EST,11:13 AM MST,8.0,126.6,10:54 AM MST,"Inside City of PHX, North of Airport",Pilot,Confirmation to ATC,"""Delta 810, down to 4000.""",NaN,Response to Direction from ATC
3,NaN,DAL,810.0,N10SDX,DTW,PHX,8:54 AM EST,11:13 AM MST,8.0,126.6,10:55 AM MST,"Inside City of PHX, North of Airport",ATC,Direction to Turn from ATC,"""Delta 810, maintain 3000, turn left heading 1...",NaN,Direction to Turn
4,NaN,DAL,810.0,N10SDX,DTW,PHX,8:54 AM EST,11:13 AM MST,8.0,126.6,10:55 AM MST,"Inside City of PHX, North of Airport",Pilot,Confirmation to ATC,"""3000. Left 1870. Delta 810.""",NaN,Response to Direction from ATC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,NaN,SWA,556.0,NaN,BWI,LAS,10:18AM EST,12:25PM PST,26L,135.0,12:22 PM PST,"Inside City of Las Vegas, East of Airport",Pilot,Confirmation to ATC,NaN,NaN,Response to Direction from ATC
381,NaN,SWA,556.0,NaN,BWI,LAS,10:18AM EST,12:25PM PST,26L,135.0,12:23 PM PST,"Inside City of Las Vegas, East of Airport",ATC,Direction to Descend,NaN,NaN,Direction to Descend
382,NaN,SWA,556.0,NaN,BWI,LAS,10:18AM EST,12:25PM PST,26L,135.0,12:23 PM PST,"Inside City of Las Vegas, East of Airport",Pilot,Confirmation to ATC,NaN,NaN,Response to Direction from ATC
383,NaN,SWA,556.0,NaN,BWI,LAS,10:18AM EST,12:25PM PST,26L,135.0,12:24 PM PST,"Inside City of Las Vegas, East of Airport",ATC,Direction to Contact another tower,NaN,NaN,Direction to Contact another tower
